# dbt Databricks demo


Slides and example project available at  https://github.com/ConstantinoSchillebeeckx/dbt_databricks_demo

# data build tool (dbt)

* https://www.getdbt.com/
* SQL + Jinja (and now Python!)
* lineage
* data testing
* docs

# docs

```
dbt docs generate
dbt docs serve
```

http://127.0.0.1:8080/#!/overview

# config

* project file (e.g. dbt_project.yml)
* property file (e.g. schema.yml)
* config block

```
name: 'e2'
version: '1.0.0'
config-version: 2

# This setting configures which "profile" dbt uses for this project.
profile: 'e2'

# Configuring models
models:
  +persist_docs:
    relation: true
    columns: true

  e2:
    +materialized: table
```

# SQL model

By default work is submitted as an "execution context" to either a SQLWarehouse cluster or an all-purpose cluster; here we change the default so that the model is executed as a notebook.

**e2/models/zip_ny.sql**

```
{{ config(create_notebook=True) }}

SELECT *
FROM {{ ref('uszips') }}
WHERE state_id = "NY"
```

# SQL model config

**e2/model/zip_ny.yaml**

```
version: 2

models:
  - name: zip_ny
    description: Zipcodes limited to NY
    columns:
      - name: zip
        description: The zipcode
        tests:
          - not_null
```

# Python model

**e2/models/zip_co.py**

```
def model(dbt, session):
    df = dbt.ref("uszips")
    return df.filter(df.state_id == "CO")
```



* `dbt`: A class compiled by dbt Core, unique to each model, enables you to run your Python code in the context of your dbt project and DAG.
* `session`: A class representing your data platform’s connection to the Python backend. The session is needed to read in tables as DataFrames, and to write DataFrames back to tables. In PySpark, by convention, the SparkSession is named spark, and available globally. For consistency across platforms, we always pass it into the model function as an explicit argument called session.

# Python model config

**e2/models/zip_co.py**

```
version: 2

models:
  - name: zip_co
    description: Zipcodes limited to CO
    columns:
      - name: zip
        description: The zipcode
        tests:
          - not_null
```          

# Python model

Models can be customized to execute in their own cluster; like all configurations, this can be done at the model level, in a propery file or at a project level.

**e2/models/zip_pr.py**

```
def model(dbt, session):
    dbt.config(
        submission_method="job_cluster",
        job_cluster_config={
            "spark_version": "11.3.x-scala2.12",
            "node_type_id": "i3.xlarge",
            "num_workers": 0,
            "spark_conf": {
                "spark.databricks.cluster.profile": "singleNode",
                "spark.master": "local[*, 4]",
            },
            "custom_tags": {"ResourceClass": "SingleNode"},
        },
    )

    df = dbt.ref("uszips")

    return df.filter(df.state_id == "PR")
```

# Selecting nodes

* `dbt ls --model zip_co+` select `zip_co` and all children (downstream) models
* `dbt ls --model +join` select `join` and all parent (upstream) models
* `dbt ls --model @zip_co` select `zip` and all children and parents of those children

![dag](dag.png)


# Let's run!

```
dbt run
dbt run --model model_name+3
dbt run --select tag:prod
```

# How it *might* work

* a single flow responsible for executing all dbt models
* by default this flow does `dbt run` (execute all models)
* flow accepts parameters so that we can easily rerun failed models like `dbt run failed_model+`


# Limitations

* EMR? this should work, haven't tried it yet, there is a `dbt-spark` adapter
* data tests can only be written in SQL out of the box
* SQL and Python models don't have parity
  * cannot declare a custom S3 location for Python models (SQL supports `location_root`), there's an [GH issue](https://github.com/dbt-labs/dbt-spark/issues/559) for this
  * table comments not materializing for Python models, through column level comments do